# Rigorous Evaluation for Research Paper

This notebook provides:
1. **5-Fold Cross-Validation** - Statistical validity of results
2. **Ablation Studies** - Contribution of each component (augmentation, demographics)
3. **Encoder Comparison** - Testing different sentence transformer models
4. **User-in-Loop Pipeline** - End-to-end evaluation with user confirmation
5. **Error Analysis** - Confusion patterns

**Note**: Uses checkpointing to resume interrupted runs. Delete `checkpoints/*.pkl` to restart.

In [1]:
# ==== Resume / Checkpoint Utilities ====
import os, pickle
from pathlib import Path

CKPT_DIR = Path('checkpoints')
CKPT_DIR.mkdir(exist_ok=True)

def get_cv_state_file(cv_name: str) -> Path:
    return CKPT_DIR / f'cv_state_{cv_name}.pkl'

def load_cv_state(cv_name: str) -> dict:
    state_file = get_cv_state_file(cv_name)
    if state_file.exists():
        with open(state_file, 'rb') as f:
            state = pickle.load(f)
            print(f"  Loaded checkpoint for '{cv_name}': {len(state.get('fold_results', {}))} folds completed")
            return state
    return {'fold_results': {}}

def save_cv_state(cv_name: str, state: dict):
    with open(get_cv_state_file(cv_name), 'wb') as f:
        pickle.dump(state, f)

def clear_cv_state(cv_name: str):
    state_file = get_cv_state_file(cv_name)
    if state_file.exists():
        os.remove(state_file)
        print(f"  Cleared checkpoint for '{cv_name}'")

print('Resume utilities loaded. Checkpoints stored in:', CKPT_DIR.absolute())

Resume utilities loaded. Checkpoints stored in: c:\Users\henry\Desktop\Programming\Python\Multimodal_Diagnosis\notebooks\checkpoints


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json, joblib, warnings, sys, os, time
from typing import Dict, List, Any

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

project_root = Path(os.getcwd()).parent
sys.path.insert(0, str(project_root))

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (accuracy_score, top_k_accuracy_score,
    confusion_matrix, f1_score)
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from joblib import Parallel, delayed
import multiprocessing

from models.architectures.symptom_classifier import SymptomCategoryClassifier, SymptomDiseaseClassifier

print(f"Project root: {project_root}")

Project root: c:\Users\henry\Desktop\Programming\Python\Multimodal_Diagnosis


---
# Part 1: Load Data

In [3]:
# Data paths
base_data_path = project_root / "data" / "processed" / "symptoms" / "symptoms_to_disease_cleaned.csv"
augmented_no_demo_path = project_root / "data" / "processed" / "symptoms" / "symptoms_augmented_no_demographics.csv"
augmented_with_demo_path = project_root / "data" / "processed" / "symptoms" / "symptoms_augmented_with_demographics.csv"

# Symptom vocabulary
with open(project_root / "data" / "symptom_vocabulary.json") as f:
    symptom_cols = json.load(f)

# Load datasets
df_base = pd.read_csv(base_data_path)
df_augmented = pd.read_csv(augmented_no_demo_path)
df_demo = pd.read_csv(augmented_with_demo_path)

print(f"Base dataset: {len(df_base):,} rows, {df_base['diseases'].nunique()} diseases")
print(f"Augmented dataset: {len(df_augmented):,} rows, {df_augmented['diseases'].nunique()} diseases")
print(f"Augmented + Demographics: {len(df_demo):,} rows, {df_demo['diseases'].nunique()} diseases")

Base dataset: 206,267 rows, 627 diseases
Augmented dataset: 207,518 rows, 627 diseases
Augmented + Demographics: 207,518 rows, 627 diseases


In [4]:
def filter_min_samples(df, min_samples=5):
    """Filter to keep only diseases with at least min_samples."""
    disease_counts = df['diseases'].value_counts()
    valid_diseases = disease_counts[disease_counts >= min_samples].index
    df_filtered = df[df['diseases'].isin(valid_diseases)].copy()
    
    print(f"Original: {len(df):,} rows, {df['diseases'].nunique()} diseases")
    print(f"Filtered: {len(df_filtered):,} rows, {df_filtered['diseases'].nunique()} diseases")
    return df_filtered

# Filter for stable evaluation
df_base_filtered = filter_min_samples(df_base)
df_augmented_filtered = filter_min_samples(df_augmented)
df_demo_filtered = filter_min_samples(df_demo)

Original: 206,267 rows, 627 diseases
Filtered: 206,173 rows, 588 diseases
Original: 207,518 rows, 627 diseases
Filtered: 207,492 rows, 616 diseases
Original: 207,518 rows, 627 diseases
Filtered: 207,492 rows, 616 diseases


---
# Part 2: Model Training Utilities

In [5]:
# Feature columns
non_feature_cols = ['diseases', 'disease_category', 'symptoms', 'age', 'sex', 'age_normalized', 'sex_encoded']

def prepare_features(df, feature_cols):
    """Prepare features and labels from dataframe."""
    available_cols = [c for c in feature_cols if c in df.columns]
    X = df[available_cols]
    y = df['diseases'].values
    return X, y, available_cols

def _train_specialist(category, X_train, y_train, y_categories):
    """Train a single specialist model (for parallel execution)."""
    mask = y_categories == category
    X_cat = X_train[mask]
    y_cat = y_train[mask]

    if len(y_cat) < 5:
        return None, None

    model = SymptomDiseaseClassifier(category=category, n_estimators=100)
    model.fit(X_cat, y_cat)
    return category, model

def train_hierarchical_model(X_train, y_train, df_train_full, n_jobs=None):
    """Train hierarchical model (Category -> Specialist Disease Classifiers)."""
    if n_jobs is None:
        n_jobs = max(1, multiprocessing.cpu_count() - 1)

    # 1. Train Category Classifier
    print("  Training Category Classifier...")
    y_categories = df_train_full['disease_category'].values

    cat_encoder = LabelEncoder()
    y_cat_encoded = cat_encoder.fit_transform(y_categories)

    cat_clf = SymptomCategoryClassifier(n_estimators=100)
    cat_clf.fit(X_train, y_cat_encoded)
    cat_clf.encoder_ = cat_encoder

    # 2. Train specialist models (PARALLEL)
    unique_categories = np.unique(y_categories)
    print(f"  Training {len(unique_categories)} Specialist Models (n_jobs={n_jobs})...")

    results = Parallel(n_jobs=n_jobs, backend="loky", verbose=0)(
        delayed(_train_specialist)(cat, X_train, y_train, y_categories)
        for cat in unique_categories
    )

    specialist_models = {cat: model for cat, model in results if model is not None}
    return cat_clf, specialist_models

def predict_hierarchical(X_test, cat_clf, specialist_models, all_possible_diseases):
    """Hierarchical prediction logic."""
    cat_probs = cat_clf.predict_proba(X_test)
    cat_encoder = cat_clf.encoder_
    
    final_probs = np.zeros((len(X_test), len(all_possible_diseases)))
    disease_to_idx = {d: i for i, d in enumerate(all_possible_diseases)}
    
    for i, cat_idx in enumerate(cat_clf.categories):
        cat_name = cat_encoder.inverse_transform([cat_idx])[0]
        
        if cat_name not in specialist_models:
            continue
        
        model = specialist_models[cat_name]
        specialist_probs = model.predict_proba(X_test)
        weight = cat_probs[:, i][:, np.newaxis]
        
        for local_idx, disease in enumerate(model.diseases):
            if disease in disease_to_idx:
                global_idx = disease_to_idx[disease]
                final_probs[:, global_idx] += weight.ravel() * specialist_probs[:, local_idx]
                
    return final_probs

def evaluate_hierarchical(X_train, X_test, y_train, y_test, df_train_full, all_disease_classes):
    """Train and evaluate using hierarchical approach."""
    cat_clf, specialist_models = train_hierarchical_model(X_train, y_train, df_train_full)
    y_proba = predict_hierarchical(X_test, cat_clf, specialist_models, all_disease_classes)
    
    y_pred_idx = np.argmax(y_proba, axis=1)
    disease_to_idx = {d: i for i, d in enumerate(all_disease_classes)}
    y_test_idx = np.array([disease_to_idx.get(d, -1) for d in y_test])
    
    valid_mask = y_test_idx != -1
    all_labels = np.arange(len(all_disease_classes))
    
    results = {
        'Top-1': accuracy_score(y_test_idx[valid_mask], y_pred_idx[valid_mask]),
        'Top-3': top_k_accuracy_score(y_test_idx[valid_mask], y_proba[valid_mask], k=3, labels=all_labels),
        'Top-5': top_k_accuracy_score(y_test_idx[valid_mask], y_proba[valid_mask], k=5, labels=all_labels),
        'Macro-F1': f1_score(y_test_idx[valid_mask], y_pred_idx[valid_mask], average='macro')
    }
    
    return results, cat_clf, specialist_models, y_proba

---# Part 2b: Qualitative Encoder Analysis

In [ ]:
from models.architectures.semantic_symptom_encoder import SemanticSymptomEncoder
# Initialize used encoder
encoder = SemanticSymptomEncoder()

test_cases = {
    "headache": ["my head is killing me", "terrible migraine", "pain in my head"],
    "shortness of breath": ["can't breathe", "difficulty breathing", "out of breath"],
    "nausea": ["feeling sick", "want to throw up", "queasy stomach"],
    "fever": ["high temperature", "running a fever", "burning up"],
    "chest pain": ["hurts in my chest", "chest pressure", "sharp chest pain"]
}

print("SEMANTIC SYMPTOM MATCHING:")
total_correct, total = 0, 0
for target, phrases in test_cases.items():
    correct_count = 0
    for p in phrases:
        res = encoder.encode_symptoms(p)
        top_syms = [s[0] for s in encoder.get_top_symptoms(res['symptom_vector'], top_k=3)]
        if any(target in s or s in target for s in top_syms):
            correct_count += 1
    total_correct += correct_count
    total += len(phrases)
    print(f"  {target}: {correct_count}/{len(phrases)}")
print(f"\nOverall Accuracy: {total_correct/total*100:.1f}%")

---
# Part 2c: Semantic Encoder Comparison (6 Models)

In [5]:
CANDIDATE_MODELS = [
    "all-mpnet-base-v2",
    "multi-qa-mpnet-base-dot-v1",
    "all-MiniLM-L12-v2",
    "paraphrase-mpnet-base-v2",
    "paraphrase-MiniLM-L6-v2",
    "sentence-transformers/msmarco-distilbert-cos-v5",
]

# Default config for model comparison
DEFAULT_THRESHOLD = 0.15
DEFAULT_EXPONENT = 1.0

In [6]:
def evaluate_encoder_on_paraphrases(encoder, paraphrases, n_tests=300, seed=42):
    """
    Evaluate encoder using natural language paraphrases.
    Returns P@5, R@5, F1@5, P@10, R@10, F1@10, MRR metrics.
    """
    random.seed(seed)
    testable = [s for s in encoder.symptoms if s in paraphrases]
    
    precision_5, recall_5 = [], []
    precision_10, recall_10 = [], []
    mrr = []
    
    for _ in range(n_tests):
        n = random.randint(2, 5)
        # Ensure we don't sample more symptoms than available
        sample_size = min(n, len(testable))
        if sample_size == 0: continue
            
        gt_symptoms = set(random.sample(testable, sample_size))
        
        phrases = [random.choice(paraphrases[s]) for s in gt_symptoms]
        text = ". ".join(phrases)
        
        result = encoder.encode_symptoms(text)
        
        # Get Top-10 for evaluation
        top_10 = encoder.get_top_symptoms(result['symptom_vector'], top_k=10, threshold=0.0)
        top_10_names = [s[0] for s in top_10]
        
        # @5 Metrics
        top_5_names = top_10_names[:5]
        hits_5 = len(set(top_5_names) & gt_symptoms)
        precision_5.append(hits_5 / 5)
        recall_5.append(hits_5 / len(gt_symptoms))
        
        # @10 Metrics
        hits_10 = len(set(top_10_names) & gt_symptoms)
        precision_10.append(hits_10 / 10)
        recall_10.append(hits_10 / len(gt_symptoms))
        
        # MRR
        for rank, (sym, _) in enumerate(top_10, 1):
            if sym in gt_symptoms:
                mrr.append(1.0 / rank)
                break
        else:
            mrr.append(0.0)
    
    # helper for safe F1
    def calc_f1(p, r): 
        if p + r == 0:
            return 0.0
        return 2 * p * r / (p + r)

    p5, r5 = np.mean(precision_5), np.mean(recall_5)
    p10, r10 = np.mean(precision_10), np.mean(recall_10)
    
    return {
        'P@5': p5, 
        'R@5': r5, 
        'F1@5': calc_f1(p5, r5),
        # Aliases for backward compatibility
        'F1': calc_f1(p5, r5), 
        'P@10': p10,
        'R@10': r10,
        'F1@10': calc_f1(p10, r10),
        'MRR': np.mean(mrr)
    }

In [7]:
# ==== Model Comparison with Checkpointing ====
print("="*60)
print("MODEL COMPARISON (with checkpointing)")
print("="*60)

state = load_checkpoint('model_comparison')
completed = state['completed']
all_results = state['results']

for model_name in CANDIDATE_MODELS:
    if model_name in completed:
        print(f"  ⏭️ {model_name}: SKIPPED (already completed)")
        continue
    
    print(f"\n▶ Testing: {model_name}")
    start = time.time()
    
    try:
        encoder = SemanticSymptomEncoder(
            model_name=model_name,
            device='cpu',
            threshold=DEFAULT_THRESHOLD,
            exponent=DEFAULT_EXPONENT
        )
        
        metrics = evaluate_encoder_on_paraphrases(encoder, paraphrases, n_tests=300)
        elapsed = time.time() - start
        
        result = {
            'model': model_name,
            'threshold': DEFAULT_THRESHOLD,
            'exponent': DEFAULT_EXPONENT,
            **metrics,
            'time': elapsed
        }
        
        all_results.append(result)
        completed[model_name] = True
        
        state['completed'] = completed
        state['results'] = all_results
        save_checkpoint('model_comparison', state)
        
        print(f"   P@5: {metrics['P@5']*100:.1f}% | R@5: {metrics['R@5']*100:.1f}% | F1: {metrics['F1']*100:.1f}% ({elapsed:.1f}s)")
        
        del encoder
        
    except Exception as e:
        print(f"   ❌ Failed: {e}")

# Summary
print("\n" + "="*60)
print("MODEL COMPARISON RESULTS")
print("="*60)
results_sorted = sorted(all_results, key=lambda x: x['F1'], reverse=True)
for r in results_sorted:
    print(f"  {r['model']:42s} P@5={r['P@5']*100:5.1f}% R@5={r['R@5']*100:5.1f}% F1={r['F1']*100:5.1f}%")

BEST_MODEL = results_sorted[0]['model']
print(f"\n🏆 Best Model: {BEST_MODEL}")

MODEL COMPARISON (with checkpointing)
  ✅ Loaded checkpoint 'model_comparison': 6 items completed
  ⏭️ all-mpnet-base-v2: SKIPPED (already completed)
  ⏭️ multi-qa-mpnet-base-dot-v1: SKIPPED (already completed)
  ⏭️ all-MiniLM-L12-v2: SKIPPED (already completed)
  ⏭️ paraphrase-mpnet-base-v2: SKIPPED (already completed)
  ⏭️ paraphrase-MiniLM-L6-v2: SKIPPED (already completed)
  ⏭️ sentence-transformers/msmarco-distilbert-cos-v5: SKIPPED (already completed)

MODEL COMPARISON RESULTS
  multi-qa-mpnet-base-dot-v1                 P@5= 34.9% R@5= 51.4% F1= 41.6%
  all-MiniLM-L12-v2                          P@5= 27.0% R@5= 40.1% F1= 32.3%
  paraphrase-mpnet-base-v2                   P@5= 26.7% R@5= 38.6% F1= 31.5%
  paraphrase-MiniLM-L6-v2                    P@5= 24.0% R@5= 35.2% F1= 28.6%
  sentence-transformers/msmarco-distilbert-cos-v5 P@5= 17.4% R@5= 25.3% F1= 20.6%
  all-mpnet-base-v2                          P@5= 10.1% R@5= 14.2% F1= 11.8%

🏆 Best Model: multi-qa-mpnet-base-dot-v1


---
# Part 3: 5-Fold Cross-Validation

In [6]:
def cross_validate(X, y, df_full, disease_classes, n_folds=5, cv_name='default'):
    """Perform stratified k-fold CV with checkpointing."""
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    cv_state = load_cv_state(cv_name)
    fold_results_dict = cv_state.get('fold_results', {})
    
    indices = np.arange(len(y))
    
    for fold, (train_idx_cv, test_idx_cv) in enumerate(skf.split(indices, y)):
        if fold in fold_results_dict:
            print(f"  Fold {fold+1}: SKIPPED (Top-1={fold_results_dict[fold]['Top-1']*100:.2f}%)")
            continue
        
        X_train_cv = X.iloc[train_idx_cv]
        X_test_cv = X.iloc[test_idx_cv]
        y_train_cv = y[train_idx_cv]
        y_test_cv = y[test_idx_cv]
        df_train_cv = df_full.iloc[train_idx_cv]
        
        results, _, _, _ = evaluate_hierarchical(
            X_train_cv, X_test_cv, y_train_cv, y_test_cv, df_train_cv, disease_classes
        )
        
        fold_results_dict[fold] = results
        cv_state['fold_results'] = fold_results_dict
        save_cv_state(cv_name, cv_state)
        
        print(f"  Fold {fold+1}: Top-1={results['Top-1']*100:.2f}% (saved)")
    
    fold_results = [fold_results_dict[i] for i in sorted(fold_results_dict.keys())]
    
    summary = {}
    for metric in fold_results[0].keys():
        values = [r[metric] for r in fold_results]
        summary[metric] = {'mean': np.mean(values), 'std': np.std(values)}
    
    print(f"  CV complete: {len(fold_results)} folds aggregated")
    return summary

In [7]:
# Prepare feature columns
feature_cols_base = [c for c in df_base.columns if c not in non_feature_cols]
feature_cols_aug = [c for c in df_augmented_filtered.columns if c not in non_feature_cols]

# For demographics, add age_normalized and sex_encoded
df_demo_filtered['sex_encoded'] = (df_demo_filtered['sex'] == 'M').astype(int)
df_demo_filtered['age_normalized'] = df_demo_filtered['age'] / 100.0
feature_cols_demo = feature_cols_aug + ['age_normalized', 'sex_encoded']
feature_cols_demo = [c for c in feature_cols_demo if c in df_demo_filtered.columns]

print(f"Base features: {len(feature_cols_base)}")
print(f"Augmented features: {len(feature_cols_aug)}")
print(f"Demo features: {len(feature_cols_demo)}")

Base features: 375
Augmented features: 456
Demo features: 458


In [8]:
# ==== Run 5-Fold CV on Base Data ====
print("="*60)
print("5-FOLD CV ON BASE/REAL DATA")
print("="*60)

X_base, y_base, _ = prepare_features(df_base_filtered, feature_cols_base)
all_diseases_base = np.unique(y_base)

cv_results_base = cross_validate(X_base, y_base, df_base_filtered, all_diseases_base, cv_name='base')

print(f"\nCross-Validation Results (Base):")
for metric, stats in cv_results_base.items():
    print(f"  {metric}: {stats['mean']*100:.2f}% ± {stats['std']*100:.2f}%")

5-FOLD CV ON BASE/REAL DATA
  Loaded checkpoint for 'base': 5 folds completed
  Fold 1: SKIPPED (Top-1=82.04%)
  Fold 2: SKIPPED (Top-1=81.84%)
  Fold 3: SKIPPED (Top-1=81.51%)
  Fold 4: SKIPPED (Top-1=82.09%)
  Fold 5: SKIPPED (Top-1=81.65%)
  CV complete: 5 folds aggregated

Cross-Validation Results (Base):
  Top-1: 81.83% ± 0.22%
  Top-3: 94.07% ± 0.06%
  Top-5: 96.54% ± 0.05%
  Macro-F1: 70.53% ± 0.32%


In [9]:
# ==== Run 5-Fold CV on Augmented Data ====
print("="*60)
print("5-FOLD CV ON AUGMENTED DATA")
print("="*60)

X_aug, y_aug, _ = prepare_features(df_augmented_filtered, feature_cols_aug)
all_diseases_aug = np.unique(y_aug)

cv_results_aug = cross_validate(X_aug, y_aug, df_augmented_filtered, all_diseases_aug, cv_name='augmented')

print(f"\nCross-Validation Results (Augmented):")
for metric, stats in cv_results_aug.items():
    print(f"  {metric}: {stats['mean']*100:.2f}% ± {stats['std']*100:.2f}%")

5-FOLD CV ON AUGMENTED DATA
  Training Category Classifier...
Training SymptomCategoryClassifier with shape (165993, 456)
  Training 14 Specialist Models (n_jobs=3)...
  Fold 1: Top-1=81.55% (saved)
  Training Category Classifier...
Training SymptomCategoryClassifier with shape (165993, 456)
  Training 14 Specialist Models (n_jobs=3)...
  Fold 2: Top-1=81.75% (saved)
  Training Category Classifier...
Training SymptomCategoryClassifier with shape (165994, 456)
  Training 14 Specialist Models (n_jobs=3)...
  Fold 3: Top-1=81.68% (saved)
  Training Category Classifier...
Training SymptomCategoryClassifier with shape (165994, 456)
  Training 14 Specialist Models (n_jobs=3)...
  Fold 4: Top-1=81.65% (saved)
  Training Category Classifier...
Training SymptomCategoryClassifier with shape (165994, 456)
  Training 14 Specialist Models (n_jobs=3)...
  Fold 5: Top-1=81.64% (saved)
  CV complete: 5 folds aggregated

Cross-Validation Results (Augmented):
  Top-1: 81.65% ± 0.07%
  Top-3: 93.81% ± 0.

In [10]:
# ==== Run 5-Fold CV on Demographics Data ====
print("="*60)
print("5-FOLD CV ON DEMOGRAPHICS DATA")
print("="*60)

X_demo = df_demo_filtered[feature_cols_demo]
y_demo = df_demo_filtered['diseases'].values
all_diseases_demo = np.unique(y_demo)

cv_results_demo = cross_validate(X_demo, y_demo, df_demo_filtered, all_diseases_demo, cv_name='demographics')

print(f"\nCross-Validation Results (Demographics):")
for metric, stats in cv_results_demo.items():
    print(f"  {metric}: {stats['mean']*100:.2f}% ± {stats['std']*100:.2f}%")

# Demographics contribution
print(f"\n📊 Demographics Contribution:")
demo_effect = (cv_results_demo['Top-1']['mean'] - cv_results_aug['Top-1']['mean']) * 100
print(f"  Demographics Effect: {demo_effect:+.2f}%")

5-FOLD CV ON DEMOGRAPHICS DATA
  Training Category Classifier...
Training SymptomCategoryClassifier with shape (165993, 458)
  Training 14 Specialist Models (n_jobs=3)...
  Fold 1: Top-1=83.65% (saved)
  Training Category Classifier...
Training SymptomCategoryClassifier with shape (165993, 458)
  Training 14 Specialist Models (n_jobs=3)...
  Fold 2: Top-1=84.18% (saved)
  Training Category Classifier...
Training SymptomCategoryClassifier with shape (165994, 458)
  Training 14 Specialist Models (n_jobs=3)...
  Fold 3: Top-1=84.01% (saved)
  Training Category Classifier...
Training SymptomCategoryClassifier with shape (165994, 458)
  Training 14 Specialist Models (n_jobs=3)...
  Fold 4: Top-1=83.94% (saved)
  Training Category Classifier...
Training SymptomCategoryClassifier with shape (165994, 458)
  Training 14 Specialist Models (n_jobs=3)...
  Fold 5: Top-1=83.97% (saved)
  CV complete: 5 folds aggregated

Cross-Validation Results (Demographics):
  Top-1: 83.95% ± 0.17%
  Top-3: 95.05

---# Part 4: Baseline Models Comparison

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

print("\n" + "="*60)
print("BASELINE MODEL EVALUATION")
print("="*60)

# Prepare simple split for baselines (using base data)
X_b, y_b, _ = prepare_features(df_base_filtered, feature_cols_base)
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X_b, y_b, test_size=0.2, random_state=42)
print(f"Training baselines on {len(X_train_b):,} samples...")

# Logistic Regression
start = time.time()
lr_clf = LogisticRegression(max_iter=500, n_jobs=-1, random_state=42)
lr_clf.fit(X_train_b, y_train_b)
lr_acc = accuracy_score(y_test_b, lr_clf.predict(X_test_b))
print(f"Logistic Regression: {lr_acc*100:.2f}% ({time.time() - start:.1f}s)")

# Random Forest
start = time.time()
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=20, n_jobs=-1, random_state=42)
rf_clf.fit(X_train_b, y_train_b)
rf_acc = accuracy_score(y_test_b, rf_clf.predict(X_test_b))
print(f"Random Forest: {rf_acc*100:.2f}% ({time.time() - start:.1f}s)")

# Compare with our best model (Demographics CV Average)
our_acc = cv_results_demo['Top-1']['mean']
print(f"\nOurs (Hierarchical + Demo): {our_acc*100:.2f}%")
print(f"Improvement over LR: +{(our_acc-lr_acc)*100:.1f}%")
print(f"Improvement over RF: +{(our_acc-rf_acc)*100:.1f}%")

---# Part 5: Statistical Significance (McNemar's Test)

In [ ]:
from scipy.stats import chi2

# Prepare data for hold-out comparison
X_s, y_s, _ = prepare_features(df_augmented_filtered, feature_cols_aug)
X_d, y_d, _ = prepare_features(df_demo_filtered, feature_cols_demo)

# Align indices (intersection)
common_idx = df_augmented_filtered.index.intersection(df_demo_filtered.index)
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_s.loc[common_idx], y_s[common_idx], test_size=0.1, random_state=42)
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_d.loc[common_idx], y_d[common_idx], test_size=0.1, random_state=42)

# Train models on single split for direct comparison
print("Training models for McNemar's test...")
model_s = SymptomDiseaseClassifier(n_estimators=50) # Faster for stat test
model_s.fit(X_train_s, y_train_s)
pred_s = model_s.predict(X_test_s)

model_d = SymptomDiseaseClassifier(n_estimators=50)
model_d.fit(X_train_d, y_train_d)
pred_d = model_d.predict(X_test_d)

# Calculate Contingency Table
correct_s = (pred_s == y_test_s)
correct_d = (pred_d == y_test_d)

b = ((~correct_s) & correct_d).sum()  # symptoms wrong, demo right
c = (correct_s & (~correct_d)).sum()  # symptoms right, demo wrong

print(f"\nMcNemar's Test Results:")
print(f"  Symptoms Accuracy: {correct_s.mean()*100:.2f}%")
print(f"  +Demo Accuracy: {correct_d.mean()*100:.2f}%")
print(f"  Disagreement: b={b}, c={c}")

if b + c > 0:
    mcnemar_stat = (abs(b - c) - 1)**2 / (b + c)
    p_value = 1 - chi2.cdf(mcnemar_stat, df=1)
    print(f"  Chi-sq Statistic: {mcnemar_stat:.2f}")
    print(f"  P-value: {p_value:.6f}")
    print(f"  Significant (p<0.05): {'Yes' if p_value < 0.05 else 'No'}")

---# Part 6: Literature Comparison

In [ ]:
literature = pd.DataFrame({
    'System': ['Ada Health', 'Babylon Health', 'Isabel Healthcare', 'WebMD', 'Ours (Hierarchical CV)'],
    'Diseases': ['~1000', '~500', '~6000', '~200', str(len(all_diseases_demo))],
    'Top-1': ['51%*', '60%*', '48%*', '35%*', f"{cv_results_demo['Top-1']['mean']*100:.1f}%"],
    'Top-5': ['70%*', '80%*', '74%*', '58%*', f"{cv_results_demo['Top-5']['mean']*100:.1f}%"]
})
print("LITERATURE COMPARISON (* = approximate from published evaluations):")
print(literature.to_string(index=False))

---
# Part 7: Save Results

In [11]:
# Save CV results
results_path = project_root / 'notebooks' / 'figures' / 'rigorous_eval_results.json'

# Load existing or create new
if results_path.exists():
    with open(results_path, 'r') as f:
        final_results = json.load(f)
else:
    final_results = {}

# Update CV results
final_results['cv_real'] = {k: {'mean': v['mean'], 'std': v['std']} for k, v in cv_results_base.items()}
final_results['cv_augmented'] = {k: {'mean': v['mean'], 'std': v['std']} for k, v in cv_results_aug.items()}
final_results['cv_augmented_demo'] = {k: {'mean': v['mean'], 'std': v['std']} for k, v in cv_results_demo.items()}

with open(results_path, 'w') as f:
    json.dump(final_results, f, indent=2, default=float)

print(f"✅ Results saved to: {results_path}")

✅ Results saved to: c:\Users\henry\Desktop\Programming\Python\Multimodal_Diagnosis\notebooks\figures\rigorous_eval_results.json


---
# Part 8: Summary Table

In [12]:
# Summary
print("\n" + "="*70)
print("ABLATION STUDY SUMMARY (5-Fold Cross-Validation)")
print("="*70)

ablation_cv_df = pd.DataFrame({
    'Configuration': ['Base (Cleaned)', 'Augmented', 'Augmented + Demographics'],
    'Top-1 (Mean ± Std)': [
        f"{cv_results_base['Top-1']['mean']*100:.2f}% ± {cv_results_base['Top-1']['std']*100:.2f}%",
        f"{cv_results_aug['Top-1']['mean']*100:.2f}% ± {cv_results_aug['Top-1']['std']*100:.2f}%",
        f"{cv_results_demo['Top-1']['mean']*100:.2f}% ± {cv_results_demo['Top-1']['std']*100:.2f}%"
    ],
    'Top-5 Mean': [
        f"{cv_results_base['Top-5']['mean']*100:.2f}%",
        f"{cv_results_aug['Top-5']['mean']*100:.2f}%",
        f"{cv_results_demo['Top-5']['mean']*100:.2f}%"
    ],
    'Macro-F1 Mean': [
        f"{cv_results_base['Macro-F1']['mean']*100:.2f}%",
        f"{cv_results_aug['Macro-F1']['mean']*100:.2f}%",
        f"{cv_results_demo['Macro-F1']['mean']*100:.2f}%"
    ]
})

print(ablation_cv_df.to_string(index=False))


ABLATION STUDY SUMMARY (5-Fold Cross-Validation)
           Configuration Top-1 (Mean ± Std) Top-5 Mean Macro-F1 Mean
          Base (Cleaned)     81.83% ± 0.22%     96.54%        70.53%
               Augmented     81.65% ± 0.07%     96.36%        70.75%
Augmented + Demographics     83.95% ± 0.17%     97.16%        72.55%


---
# Part 9: User-in-Loop Pipeline Evaluation

In [8]:
# Load hierarchical models for pipeline evaluation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from models.architectures.symptom_classifier import SymptomCategoryClassifier, SymptomDiseaseClassifier
from joblib import Parallel, delayed
import multiprocessing

# Prepare data
non_feature_cols = ['diseases', 'disease_category', 'symptoms', 'age', 'sex', 'age_normalized', 'sex_encoded']
feature_cols = [c for c in df.columns if c not in non_feature_cols]

# Filter to diseases with enough samples
disease_counts = df['diseases'].value_counts()
valid_diseases = disease_counts[disease_counts >= 5].index
df_filtered = df[df['diseases'].isin(valid_diseases)].copy()

print(f"Filtered: {len(df_filtered):,} rows, {df_filtered['diseases'].nunique()} diseases")

Filtered: 207,492 rows, 616 diseases


In [9]:
def _train_specialist(category, X_train, y_train, y_categories):
    mask = y_categories == category
    X_cat = X_train[mask]
    y_cat = y_train[mask]
    if len(y_cat) < 5:
        return None, None
    model = SymptomDiseaseClassifier(category=category, n_estimators=100)
    model.fit(X_cat, y_cat)
    return category, model

def train_hierarchical_model(X_train, y_train, df_train):
    """Train hierarchical model."""
    n_jobs = max(1, multiprocessing.cpu_count() - 1)
    
    y_categories = df_train['disease_category'].values
    cat_encoder = LabelEncoder()
    y_cat_encoded = cat_encoder.fit_transform(y_categories)
    
    cat_clf = SymptomCategoryClassifier(n_estimators=100)
    cat_clf.fit(X_train, y_cat_encoded)
    cat_clf.encoder_ = cat_encoder
    
    unique_categories = np.unique(y_categories)
    results = Parallel(n_jobs=n_jobs, backend="loky", verbose=0)(
        delayed(_train_specialist)(cat, X_train, y_train, y_categories)
        for cat in unique_categories
    )
    
    specialist_models = {cat: model for cat, model in results if model is not None}
    return cat_clf, specialist_models

def predict_hierarchical(X_test, cat_clf, specialist_models, all_diseases):
    cat_probs = cat_clf.predict_proba(X_test)
    cat_encoder = cat_clf.encoder_
    
    final_probs = np.zeros((len(X_test), len(all_diseases)))
    disease_to_idx = {d: i for i, d in enumerate(all_diseases)}
    
    for i, cat_idx in enumerate(cat_clf.categories):
        cat_name = cat_encoder.inverse_transform([cat_idx])[0]
        if cat_name not in specialist_models:
            continue
        model = specialist_models[cat_name]
        specialist_probs = model.predict_proba(X_test)
        weight = cat_probs[:, i][:, np.newaxis]
        
        for local_idx, disease in enumerate(model.diseases):
            if disease in disease_to_idx:
                global_idx = disease_to_idx[disease]
                final_probs[:, global_idx] += weight.ravel() * specialist_probs[:, local_idx]
    
    return final_probs

In [10]:
# Train hierarchical model (needed for pipeline eval)
print("Training hierarchical classifier...")

X = df_filtered[feature_cols].fillna(0)
y = df_filtered['diseases'].values
all_diseases = np.unique(y)

indices = np.arange(len(df_filtered))
train_idx, test_idx = train_test_split(indices, test_size=0.1, random_state=42, stratify=y)

X_train = X.iloc[train_idx]
y_train = y[train_idx]
df_train = df_filtered.iloc[train_idx]

cat_clf, specialist_models = train_hierarchical_model(X_train, y_train, df_train)
print(f"Trained {len(specialist_models)} specialist models")

Training hierarchical classifier...
Training SymptomCategoryClassifier with shape (186742, 456)
Trained 14 specialist models


In [11]:
def evaluate_user_in_loop(
    encoder, X_true, y_true, symptom_cols, cat_clf, specialist_models, 
    all_diseases, paraphrases, top_k=20, max_samples=2500
):
    """Simulate user-in-loop pipeline WITH PARAPHRASES."""
    # Create bidirectional mapping between symptom_cols and encoder symptoms
    col_to_symptom = {}
    symptom_to_col_idx = {}
    
    for col_idx, col in enumerate(symptom_cols):
        normalized = encoder._normalize(col.replace("_", " "))
        col_to_symptom[col] = normalized
        if normalized in encoder.symptom_to_idx:
            symptom_to_col_idx[normalized] = col_idx
    
    np.random.seed(42)
    idxs = np.random.choice(len(X_true), size=min(max_samples, len(X_true)), replace=False)
    
    y_true_idx = []
    y_pred_proba = []
    disease_to_idx = {d: i for i, d in enumerate(all_diseases)}
    
    for i in idxs:
        # Get ground truth symptoms
        active_cols = [col for j, col in enumerate(symptom_cols) if X_true[i, j] == 1.0]
        if len(active_cols) < 2:
            continue
        
        # ✅ USE PARAPHRASES, not exact names!
        active_syms = [col_to_symptom[c] for c in active_cols]
        phrases = []
        for sym in active_syms:
            if sym in paraphrases:
                phrases.append(random.choice(paraphrases[sym]))
            else:
                phrases.append(sym)  # fallback
        text = ". ".join(phrases)
        
        # Encode
        result = encoder.encode_symptoms(text)
        symptom_vector = result["symptom_vector"]
        
        # Get detected symptoms using encoder's threshold
        # This time, threshold REALLY matters because paraphrases might have lower similarity
        top_results = encoder.get_top_symptoms(symptom_vector, top_k=100, threshold=encoder.threshold)
        detected_syms = {sym for sym, score in top_results}
        
        # Simulate user confirmation with false positive rejection
        X_sim = np.zeros(len(symptom_cols))
        confirmed_count = 0
        
        for sym_name in detected_syms:
            if sym_name in active_syms and sym_name in symptom_to_col_idx:
                # ✅ User confirms true positive
                col_idx = symptom_to_col_idx[sym_name]
                X_sim[col_idx] = 1.0
                confirmed_count += 1
            # Note: We're NOT adding false positives (user rejects them)
        
        # Skip if no symptoms confirmed
        if confirmed_count == 0:
            # Maybe use top-1 symptom as fallback?
            continue
        
        # Predict
        proba = predict_hierarchical(
            X_sim.reshape(1, -1), cat_clf, specialist_models, all_diseases
        )[0]
        
        y_pred_proba.append(proba)
        y_true_idx.append(disease_to_idx[y_true[i]])
    
    if len(y_pred_proba) == 0:
        return {'top1': 0.0, 'top3': 0.0, 'top5': 0.0, 'samples': 0}
    
    y_pred_proba = np.array(y_pred_proba)
    y_true_idx = np.array(y_true_idx)
    
    top1 = accuracy_score(y_true_idx, np.argmax(y_pred_proba, axis=1))
    top3 = top_k_accuracy_score(y_true_idx, y_pred_proba, k=3, labels=np.arange(len(all_diseases)))
    top5 = top_k_accuracy_score(y_true_idx, y_pred_proba, k=5, labels=np.arange(len(all_diseases)))
    
    return {'top1': top1, 'top3': top3, 'top5': top5, 'samples': len(y_true_idx)}

In [15]:
# ==== User-in-Loop Pipeline Sweep with Checkpointing ====
print("="*60)
print("USER-IN-LOOP PIPELINE EVALUATION (with checkpointing)")
print("="*60)

# Clear checkpoint because evaluation logic changed
clear_checkpoint('pipeline_sweep')
state = load_checkpoint('pipeline_sweep')
completed = state['completed']
pipeline_results = state['results']

# Pre-compute X for speed
X_values = df_filtered[feature_cols].fillna(0).values
y_values = df_filtered['diseases'].values

# Full parameter space
PIPELINE_THRESHOLDS = [0.15, 0.25, 0.35, 0.40, 0.45, 0.50]
PIPELINE_EXPONENTS = [0.5, 1.0, 1.5, 2.0, 2.5]

# Build list of configs to run
configs_to_run = []

for thresh in PIPELINE_THRESHOLDS:
    for exp in PIPELINE_EXPONENTS:
        config_key = f"{thresh}_{exp}"
        
        # Skip if already completed (checkpoint takes priority)
        if config_key in completed:
            continue
        
        configs_to_run.append((thresh, exp))

total = len(PIPELINE_THRESHOLDS) * len(PIPELINE_EXPONENTS)
print(f"Total configs: {total}, Already completed: {len(completed)}, To run: {len(configs_to_run)}")

# ✅ Use the BEST model from Part 1, not hardcoded
base_encoder = SemanticSymptomEncoder(
    model_name="multi-qa-mpnet-base-dot-v1",  # <-- Changed from "all-mpnet-base-v2"
    threshold=0.15,
    exponent=1.0,
    symptom_list=feature_cols
)

print(f"Using model: {BEST_MODEL}")

for (thresh, exp) in configs_to_run:
    config_key = f"{thresh}_{exp}"
    
    print(f"\n▶ Testing thresh={thresh}, exp={exp}...")
    
    # Update encoder parameters
    base_encoder.threshold = thresh
    base_encoder.exponent = exp
    
    # Quick debug to verify parameters work
    test_text = "headache and fever"
    result = base_encoder.encode_symptoms(test_text)
    symptom_vector = result["symptom_vector"]
    
    # Count non-zero symptoms
    non_zero = np.sum(symptom_vector > 0.0)
    print(f"  Non-zero symptoms with thresh={thresh}: {non_zero}")
    
    # Run the actual evaluation
    metrics = evaluate_user_in_loop(
        base_encoder, X_values, y_values, feature_cols,
        cat_clf, specialist_models, all_diseases,
        paraphrases,  # <-- Fixed: was 'paraphrase'
        top_k=20, max_samples=500
    )
    
    result = {
        'threshold': thresh,
        'exponent': exp,
        **metrics
    }
    
    pipeline_results.append(result)
    completed[config_key] = True
    
    state['completed'] = completed
    state['results'] = pipeline_results
    save_checkpoint('pipeline_sweep', state)
    
    print(f"  Top-1={metrics['top1']*100:.1f}%, Top-3={metrics['top3']*100:.1f}%, Top-5={metrics['top5']*100:.1f}%")

# Clean up
del base_encoder

# Best result
if pipeline_results:
    best_pipeline = max(pipeline_results, key=lambda x: x['top1'])
    print(f"\n🏆 Best Pipeline Config: threshold={best_pipeline['threshold']}, exponent={best_pipeline['exponent']}")
    print(f"   Top-1={best_pipeline['top1']*100:.1f}%, Top-3={best_pipeline['top3']*100:.1f}%, Top-5={best_pipeline['top5']*100:.1f}%")

USER-IN-LOOP PIPELINE EVALUATION (with checkpointing)
  Cleared checkpoint 'pipeline_sweep'
Total configs: 30, Already completed: 0, To run: 30
[Encoder] Loading model: multi-qa-mpnet-base-dot-v1
[Encoder] Loaded cached symptom embeddings
[Encoder] Initialized with 456 symptoms
Using model: multi-qa-mpnet-base-dot-v1

▶ Testing thresh=0.15, exp=0.5...
  Non-zero symptoms with thresh=0.15: 456
  Top-1=69.0%, Top-3=85.1%, Top-5=89.5%

▶ Testing thresh=0.15, exp=1.0...
  Non-zero symptoms with thresh=0.15: 456
  Top-1=67.5%, Top-3=84.7%, Top-5=89.1%

▶ Testing thresh=0.15, exp=1.5...
  Non-zero symptoms with thresh=0.15: 456
  Top-1=67.0%, Top-3=83.5%, Top-5=87.7%

▶ Testing thresh=0.15, exp=2.0...
  Non-zero symptoms with thresh=0.15: 456
  Top-1=63.5%, Top-3=81.0%, Top-5=86.5%

▶ Testing thresh=0.15, exp=2.5...
  Non-zero symptoms with thresh=0.15: 456
  Top-1=60.6%, Top-3=80.2%, Top-5=84.3%

▶ Testing thresh=0.25, exp=0.5...
  Non-zero symptoms with thresh=0.25: 453
  Top-1=66.7%, Top-